In [1]:
from datetime import datetime

import time
import os
import sys
import itertools
from pathlib import Path

import numpy as np
from scipy.stats import lognorm
import pandas as pd

from astropy import stats
from astropy.io import fits
from astropy.time import Time
import astropy.units as u

import matplotlib
#matplotlib.use('nbagg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

config = tf.ConfigProto(
    intra_op_parallelism_threads=8,
    inter_op_parallelism_threads=8,
    log_device_placement=True
)
sess = tf.Session(config=config)

np.random.seed(42)

print(tf.__version__)

1.14.0


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [8]:
def plot_history(history, label=None):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error')
    plt.plot(hist['epoch'], hist['mean_absolute_error'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
           label = 'Val Error')
    plt.legend()
    if label is not None:
        plt.title(label)

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error')
    plt.plot(hist['epoch'], hist['mean_squared_error'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_mean_squared_error'],
           label = 'Val Error')
    plt.legend()
    if label is not None:
        plt.title(label)
    plt.savefig(f"{label}_train.pdf")
    plt.show()

def build_model(width=32, shape=1, nlayers=1, tolerance=0.001):
    nn_layers = [layers.Dense(width, activation=tf.nn.relu, input_shape=[shape])]  # input_shape=[len(train_dataset.keys())])]

    if nlayers > 0:
        for i in range(nlayers-1):
            nn_layers.append(layers.Dense(width, activation=tf.nn.relu))
    
    nn_layers.append(layers.Dense(1))
    model = keras.Sequential(nn_layers)
    optimizer = tf.keras.optimizers.RMSprop(tolerance)

    model.compile(
        loss='mean_squared_error',
        optimizer=optimizer,
        metrics=['mean_absolute_error', 'mean_squared_error']
    )
    return model

def load_wfs_data(files=[f"../raw_data/2018_2019_wfs_loose.csv"]):
    # load data and make some subsets for each wfs for inspection later
    dfs = []
    for f in files:
        dfs.append(pd.read_csv(f))
    data = pd.concat(dfs)
    data['ut'] = pd.to_datetime(data.ut)
    data['az'][data['az'] < 0.] += 360.

    f9 = data[(data['wfs'] == 'newf9') | (data['wfs'] == 'oldf9')]
    f5 = data[data['wfs'] == 'f5']
    mmirs = data[data['wfs'] == 'mmirs']
    bino = data[data['wfs'] == 'binospec']

    # wrangle the times to add colums for mjd to look for trends over time and hour to look for nightly trends
    raw_times = data['time']
    times = Time(raw_times.values.tolist(), format='isot', scale='utc')
    mjd = times.mjd
    data['mjd'] = mjd.tolist()
    data['hour'] = data['ut'].dt.hour

    e_series = pd.read_csv("../halcoll/data/halcoll_temps.csv")

    fixed = data.drop(columns=['ut']).set_index(pd.DatetimeIndex(data['time'], name='ut'))

    e_series = e_series.set_index(pd.DatetimeIndex(e_series['ts'], name='ut').tz_localize('MST').tz_convert(None)).drop(columns=['ts'])

    merged = pd.merge_asof(fixed.sort_index(), e_series, on='ut')

    # trim out columns not relevant to training
    dropped_cols = [
        'ut', 'time', 'airmass', 'cc_x_err', 'cc_y_err', 'chamt', 'osst',
        'outt', 'exptime', 'file', 'focerr', 'fwhm', 'raw_seeing', 'residual_rms',
        'seeing', 'wavefront_rms', 'xcen', 'ycen', 'comaerr'
    ]
    trimmed = merged.drop(columns=dropped_cols)
    trimmed = trimmed.dropna()
    return trimmed

def norm_dataset(x, s):
    return (x - s['mean']) / s['std']

def unnorm_dataset(x, s):
    return x * s['std'] + s['mean']

In [4]:
labels = ['focus', 'tiltx', 'tilty', 'transx', 'transy']
trimmed = load_wfs_data(files=["../raw_data/2017_wfs.csv", "../raw_data/2018_wfs.csv"])

# split data set up into three training sets: f9, mmirs, and f5/bino. these sets correspond to the three optical configurations
# we use: f/9 and no corrector, f/5 with no corrector (mmirs), and f/5 with spectroscopic corrector (f5/bino).
datasets = {}
wfs = trimmed.pop('wfs')
datasets['f9'] = trimmed[(wfs == 'newf9') | (wfs == 'oldf9')]
datasets['mmirs'] = trimmed[wfs == 'mmirs']
datasets['f5'] = trimmed[(wfs == 'f5') | (wfs == 'binospec')]

# the large range in offsets is messing up the training. so normalize the hexapod coords to their means
means = {}
train_datasets = {}
test_datasets = {}
train_labels = {}
test_labels = {}
train_stats = {}
normed_train_data = {}
normed_test_data = {}
for w in datasets:
    means[w] = {}
    for l in labels:
        means[w][l] = datasets[w][l].mean()
        datasets[w][l] -= means[w][l]

    train_datasets[w] = datasets[w].sample(frac=0.75, random_state=0)
    #train_datasets[w] = datasets[w][datasets[w]['mjd'] < 58484.]
    test_datasets[w] = datasets[w].drop(train_datasets[w].index)
    train_stats[w] = train_datasets[w].describe()
    train_stats[w] = train_stats[w].drop(columns=labels)
    train_stats[w] = train_stats[w].transpose()
    
    train_labels[w] = {}
    test_labels[w] = {}
    for l in labels:
        train_labels[w][l] = train_datasets[w].pop(l)
        test_labels[w][l] = test_datasets[w].pop(l)
        
    normed_train_data[w] = norm_dataset(train_datasets[w], train_stats[w])
    normed_test_data[w] = norm_dataset(test_datasets[w], train_stats[w])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [18]:
models = {}
mod_config = {
    "f9": {
        'width': 8,
        'nlayers': 2
    },
    "f5": {
        'width': 32,
        'nlayers': 2
    },
    'mmirs': {
        'width': 32,
        'nlayers': 2
    }
}

for w in datasets:
    models[w] = {}
    for l in labels:
        models[w][l] = build_model(
            width=mod_config[w]['width'],
            shape=len(train_datasets[w].keys()),
            nlayers=mod_config[w]['nlayers']
        )

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=200)

In [19]:
sel_wfs = 'mmirs'
models[sel_wfs]['focus'].summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_60 (Dense)             (None, 32)                640       
_________________________________________________________________
dense_61 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_62 (Dense)             (None, 1)                 33        
Total params: 1,729
Trainable params: 1,729
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: 
            print('')
        print('.', end='')

EPOCHS = 4000

histories = {}

for l in labels:
    print(f"Training {l} model....")
    histories[l] = models[sel_wfs][l].fit(
        normed_train_data[sel_wfs], train_labels[sel_wfs][l],
        epochs=EPOCHS, validation_split = 0.2, verbose=0,
        callbacks=[early_stop, PrintDot()]
    )
    print("\n")

Training focus model....

....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
.................................................................

In [21]:
for l in labels:
    models[sel_wfs][l].save(f"../halcoll/data/{sel_wfs}_{l}_train_from_2017_2018_32x32_model.h5")

In [22]:
plot_history(histories['focus'], label="focus")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
def show_results(wfs, label):
    loss, mae, mse = models[wfs][label].evaluate(normed_test_data[wfs], test_labels[wfs][label], verbose=0)

    print("Testing set Mean Abs Error: {:5.2f} um".format(mae))
    print("Testing set RMS: {:5.2f} um".format(np.sqrt(mse)))

    test_predictions = models[wfs][label].predict(normed_test_data[wfs]).flatten()

    plt.figure()
    plt.scatter(test_labels[wfs][label], test_labels[wfs][label] - test_predictions)
    plt.xlabel('True Values [um]')
    plt.ylabel('Residuals [um]')
    minx, maxx = min(test_labels[wfs][label]), max(test_labels[wfs][label])
    plt.plot([minx, maxx], [0, 0])
    plt.show()

In [24]:
show_results(sel_wfs, 'focus')

Testing set Mean Abs Error: 17.05 um
Testing set RMS: 32.14 um


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [60]:
lab = 'focus'
test_predictions = models[sel_wfs][lab].predict(normed_test_data['f9']).flatten()
diff = test_labels[sel_wfs][lab] - test_predictions
diff.idxmax(), diff.idxmin()

(3040, 711)

In [ ]:
test_labels[sel_wfs][lab].idxmin()

In [ ]:
test_datasets[sel_wfs].loc[711]

In [ ]:
chk_data = normed_test_data[normed_test_data[sel_wfs] > 0]
#chk.index
chk_test = test_labels['focus'][chk_data.index] 
chk_pred = models['focus'].predict(chk_data).flatten()
chk_diff = chk_test - chk_pred
chk_diff.std()

In [ ]:
m = keras.models.load_model("/Users/tim/MMT/HALcoll/halcoll/data/tiltx_model.h5")

In [ ]:
m.evaluate(normed_test_data, test_labels['tiltx'])

In [ ]:
ttt = train_dataset['cell_e_series_chamber_ambient_C'] - train_dataset['yankee_temperature']
ttt.min()

In [17]:
test_dataset = load_wfs_data(["../raw_data/2019_wfs.csv"])
w = test_dataset.pop("wfs")
test_data = test_dataset[(w == 'newf9') | (w == 'oldf9')]
f9_test_labels = {}
for l in labels:
    test_data[l] -= means['f9'][l]
    f9_test_labels[l] = test_data.pop(l)
ntest_data = norm_dataset(test_data, train_stats['f9'])

loss, mae, mse = models['f9']['focus'].evaluate(ntest_data, f9_test_labels['focus'], verbose=0)

print("Testing set Mean Abs Error: {:5.2f} um".format(mae))
print("Testing set RMS: {:5.2f} um".format(np.sqrt(mse)))

test_predictions = models['f9']['focus'].predict(ntest_data).flatten()

plt.figure()
plt.scatter(f9_test_labels['focus'], f9_test_labels['focus'] - test_predictions)
plt.xlabel('True Values [um]')
plt.ylabel('Residuals [um]')
minx, maxx = min(f9_test_labels['focus']), max(f9_test_labels['focus'])
plt.plot([minx, maxx], [0, 0])
plt.show()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Testing set Mean Abs Error: 116.87 um
Testing set RMS: 144.68 um


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
test_dataset = load_wfs_data(["../raw_data/2019_wfs.csv"])
w = test_dataset.pop("wfs")
test_data = test_dataset[w == 'mmirs']
mmirs_test_labels = {}
for l in labels:
    test_data[l] -= means['mmirs'][l]
    mmirs_test_labels[l] = test_data.pop(l)
ntest_data = norm_dataset(test_data, train_stats['mmirs'])

loss, mae, mse = models['mmirs']['focus'].evaluate(ntest_data, mmirs_test_labels['focus'], verbose=0)

print("Testing set Mean Abs Error: {:5.2f} um".format(mae))
print("Testing set RMS: {:5.2f} um".format(np.sqrt(mse)))

test_predictions = models['mmirs']['focus'].predict(ntest_data).flatten()

plt.figure()
plt.scatter(mmirs_test_labels['focus'], mmirs_test_labels['focus'] - test_predictions)
plt.xlabel('True Values [um]')
plt.ylabel('Residuals [um]')
minx, maxx = min(mmirs_test_labels['focus']), max(mmirs_test_labels['focus'])
plt.plot([minx, maxx], [0, 0])
plt.show()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Testing set Mean Abs Error: 159.01 um
Testing set RMS: 210.09 um


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …